[Last Stone Weight II](https://leetcode.com/problems/last-stone-weight-ii/)。一堆石头，两两碰撞，质量小的完全消失，质量大的相应抵消。问最后能剩下的石头的最小质量是多少？

思路：根据石头个数的奇偶性，最后只可能剩下$0$个或$1$个石头。假设$n$个石头总质量为$sum$，题目要求个数不超过$n//2$、质量不超过$sum//2$的条件下取出石头的最大质量，即背包问题。

[Perfect Squares](https://leetcode.com/problems/perfect-squares/)。给一个数$n$，问其最少能用多少个完全平方数来求和得到。

思路：从数学角度上来说，根据四平方和定理，任意正整数都可以表示成不超过$4$个完全平方数的和，但是该定理一般人都不知道。因此使用动态规划来解。令```dp[i]```表示正整数$i$可以表示成多少个完全平方数的和，首先```dp[0]=0```，令完全平方数的基数为$1$，即只考虑$1*1$，更新dp数组，然后考虑$2*2$，再更新dp数组。

注：动态规划方法现在不能再通过了，全部超时。

In [30]:
def numSquares(n: int) -> int:
    dp = [4]*(n+1)
    dp[0] = 0

    for i in range(n+1):
        for base in range(1, int(n**0.5)+1):    # 完全平方数的基数
            new_num = i+base**2
            if new_num <= n:
                # 从dp[i]到dp[i+base**2]只加了一个完全平方数
                dp[new_num] = min(dp[new_num], dp[i]+1)

    return dp[-1]

1

[Ones and Zeroes](https://leetcode.com/problems/ones-and-zeroes/)。给定一系列只包含$0$和$1$的字串，限定$0$的个数为$m$，$1$的个数为$n$，不能重复取，问最多在字典中取出几个字串。

思路：动态规划，设立一个二维dp数组，行表示$0$的数量，列表式$1$的数量。初始```dp[0][0]=0```，将$m$与$n$看做是容量，那么类似于0-1背包问题，将大问题分解成子问题，由后往前遍历(难点)，```dp[row][col]=max(dp[row-zeros][col-ones]+1, dp[row][col])```。

In [46]:
def findMaxForm(strs, m: int, n: int) -> int:
    n_zeros, n_ones = m, n
    dp = [[0 for _ in range(n_ones+1)] for _ in range(n_zeros+1)]

    for s in strs:
        zeros = len([ch for ch in s if ch == '0'])
        ones = len([ch for ch in s if ch == '1'])

        # 由后往前更新
        for row in range(n_zeros, zeros-1, -1):
            for col in range(n_ones, ones-1, -1):
                dp[row][col] = max(dp[row-zeros][col-ones]+1, dp[row][col])

    return dp[-1][-1]

4

[Target Sum](https://leetcode.com/problems/target-sum/)。给一个长度为$n$的非负数组，以及一个目标值$S$，在数组的所有元素前选择加上```+```或```-```，使得数组和为$S$。求有多少种配置。

思路：DFS方法超时，使用DP。注意每增加一个元素，都会引入两种可能：$+x$和$-x$。对于$0$个数，可能构成的和只有$1$种：$\{0\}$；增加一个数$x_{1}$，那么可能构成的和至多有两种：$\{-x_{1},x_{1}\}$；再增加一个数$x_{2}$，可能构成的和至多有$4$种：$\{-x_{1}-x_{2},-x_{1}+x_{2},x_{1}-x_{2},x_{1}+x_{2}\}$。

题目保证原数组的和不会超过$1000$，那么考虑全部加```-```号的情况，所有可能的和范围在$[-1000,1000]$共$2001$种情况。设立二维$dp$数组，$dp[i][j]$表示使用$i$个数字构成和$j$有多少种情况。那么转态转移方程为：

$$
dp[i][j]=dp[i-1][j-nums[i]]+dp[i-1][j+nums[i]]
$$

In [9]:
def findTargetSumWays(nums, S: int) -> int:
    if not nums or abs(S) > sum(nums):
        return 0

    num_sum = sum(nums)
    col_range = 2*num_sum+1
    row_range = len(nums)+1

    dp = [[0 for _ in range(col_range)] for _ in range(row_range)]
    dp[0][num_sum] = 1

    for i in range(1, row_range):
        for j in range(col_range):
            if j-nums[i-1] >= 0:
                dp[i][j] += dp[i-1][j-nums[i-1]]
            if j+nums[i-1] < col_range:
                dp[i][j] += dp[i-1][j+nums[i-1]]

    return dp[-1][num_sum+S]

0

[Delete Operation for Two Strings](https://leetcode.com/problems/delete-operation-for-two-strings/)。给两字符串，问最少需要做多少次删除操作才能使得两字符串相等。

思路：该题是求两字符串最长公共子序列的反向题目。设一个二维数组$dp$，其中$dp[i][j]$表示字串$1$的前$i$位与字串$2$的前$j$位的最长公共子序列长度，易得$dp[0][0]=0$。不难得出状态转移方程为：

$$
dp[i][j]=
\begin{cases}
dp[i-1][j-1]+1 & str_{1}[i-1]==str_{2}[j-1] \\
max(dp[i][j-1],dp[i-1][j]) & str_{1}[i-1]{\ne}str_{2}[j-1] \\
\end{cases}
$$

原理是这样：若两字符串的某一位置相等，则LCS长度可以直接加一；若不相等，则需要取错位比较下的最大长度。

In [2]:
def minDistance(word1: str, word2: str) -> int:
    n, m = len(word1), len(word2)
    dp = [[0]*(m+1) for _ in range(n+1)]
    for i in range(1, n+1):
        for j in range(1, m+1):
            if word1[i-1] == word2[j-1]:
                dp[i][j] = dp[i-1][j-1]+1
            else:
                dp[i][j] = max(dp[i-1][j], dp[i][j-1])

    return m+n-2*dp[-1][-1]

2

[Longest Palindromic Subsequence](https://leetcode.com/problems/longest-palindromic-subsequence/)。给一字符串，求其最长回文子序列。

思路：找回文的方法就是从中心往两边扩展。因为子序列不要求连续，所以使用dp的方法来解。设一个二维数组$dp$，其中$dp[i][j]$表示$[i,j]$区间内的最长回文子序列的长度，那么易得有$0\le{i\le{j}}$，所以只使用dp数组的上三角。扩展即可得到状态转移方程：

$$
dp[i][j]=
\begin{cases}
dp[i+1][j-1]+2 & s[i]=s[j] \\
max(dp[i+1][j],dp[i][j-1]) & s[i]\ne{s[j]} \\
\end{cases}
$$

注意最后返回的答案是$dp[0][-1]$，并且填充dp数组时由下往上填充。

In [1]:
def longestPalindromeSubseq(s: str) -> int:
    n = len(s)
    if n < 2:
        return n

    dp = [[0]*n for _ in range(n)]
    for i in range(n-1, -1, -1):
        dp[i][i] = 1    # 单字符为1
        for j in range(i+1, n):
            if s[i] == s[j]:
                dp[i][j] = dp[i+1][j-1]+2
            else:
                dp[i][j] = max(dp[i+1][j], dp[i][j-1])

    return dp[0][-1]

4